In [2]:
# Basic data handling
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Set display options
pd.set_option('display.max_columns', 100)
sns.set(style="whitegrid")


In [8]:
# Load the dataset
data_path = "C:\\Users\\Cesar Dushimimana\\Documents\\aave-credit-scoring\\data\\aave_cleaned_transactions.csv"

df = pd.read_csv(data_path)

# Check basic structure
df.info()
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   userid           100000 non-null  object 
 1   timestamp        100000 non-null  object 
 2   blocknumber      100000 non-null  int64  
 3   action           100000 non-null  object 
 4   action_type      100000 non-null  object 
 5   amount           100000 non-null  float64
 6   asset_symbol     99752 non-null   object 
 7   asset_price_usd  100000 non-null  float64
 8   pool_id          99752 non-null   object 
 9   user_id_nested   100000 non-null  object 
 10  to_id            32305 non-null   object 
 11  amount_usd       100000 non-null  float64
 12  user_differs     100000 non-null  bool   
dtypes: bool(1), float64(3), int64(1), object(8)
memory usage: 9.3+ MB


,userid,timestamp,blocknumber,action,action_type,amount,asset_symbol,asset_price_usd,pool_id,user_id_nested,to_id,amount_usd,user_differs
0,0x00000000001accfa9cef68cf5371a23025b6d4b6,2021-08-17 05:29:26,1629178166,deposit,Deposit,2.000000e+09,USDC,0.993832,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0x00000000001accfa9cef68cf5371a23025b6d4b6,NaN,1.987664e+09,False
1,0x000000000051d07a4fb3bd10121a343d85818da6,2021-05-20 15:36:53,1621525013,deposit,Deposit,1.450000e+20,WMATIC,1.970307,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0x000000000051d07a4fb3bd10121a343d85818da6,NaN,2.856945e+20,False
2,0x000000000096026fb41fc39f9875d164bd82e2dc,2021-07-24 09:28:33,1627118913,deposit,Deposit,1.000000e+15,WMATIC,0.922377,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0x000000000096026fb41fc39f9875d164bd82e2dc,NaN,9.223773e+14,False
3,0x000000000096026fb41fc39f9875d164bd82e2dc,2021-07-31 23:15:18,1627773318,deposit,Deposit,4.000000e+15,WMATIC,1.057483,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0x000000000096026fb41fc39f9875d164bd82e2dc,NaN,4.229933e+15,False
4,0x0000000000e189dd664b9ab08a33c4839953852c,2021-04-19 15:25:07,1618845907,redeemunderlying,RedeemUnderlying,5.015480e+05,USDC,1.002130,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0x7f90122bf0700f9e7e1f688fe926940e8839f353,0x0000000000e189dd664b9ab08a33c4839953852c,5.026161e+05,True


### Understand the overall dataset structure and basic stats

In [28]:

# Unique users and actions count
print(f"Total unique wallets (userid): {df['userid'].nunique():,}")
print(f"Action types and counts:\n{df['action'].value_counts()}")


Total unique wallets (userid): 3,497
Action types and counts:
action
deposit             37808
redeemunderlying    32305
borrow              17086
repay               12553
liquidationcall       248
Name: count, dtype: int64


### Visualize transaction action distribution and total USD volumes per action

#### Count Transactions By Action

In [34]:
fig = px.histogram(df, x='action', title='Count of Transactions by Action Type', text_auto=True)
fig.update_traces(textposition='outside')
fig.show()


#### Top Assets by Transaction Count and USD Volume

In [39]:
import plotly.express as px

# Top 10 assets by transaction count
top_assets_count = df['asset_symbol'].value_counts().nlargest(10).reset_index()
top_assets_count.columns = ['asset_symbol', 'count']

fig = px.bar(top_assets_count, x='asset_symbol', y='count',
             title='Top 10 Assets by Transaction Count',
             labels={'asset_symbol': 'Asset Symbol', 'count': 'Transactions'},
             text='count')
fig.update_traces(textposition='outside')
fig.show()



#### Top 10 assets by total USD volume

In [43]:
import plotly.express as px

# formatted volume labels
def format_usd(value):
    if value >= 1e12:
        return f'{value / 1e12:.1f}T'
    elif value >= 1e9:
        return f'{value / 1e9:.1f}B'
    elif value >= 1e6:
        return f'{value / 1e6:.1f}M'
    else:
        return f'{value:.0f}'

top_assets_usd = df.groupby('asset_symbol')['amount_usd'].sum().nlargest(10).reset_index()
top_assets_usd['formatted_amount'] = top_assets_usd['amount_usd'].apply(format_usd)

# Plotly bar chart
fig2 = px.bar(
    top_assets_usd,
    x='asset_symbol',
    y='amount_usd',
    text='formatted_amount',
    title='Top 10 Assets by USD Volume',
    labels={'asset_symbol': 'Asset Symbol', 'amount_usd': 'Total USD Volume'},
    color='amount_usd',
    color_continuous_scale='Blues'  # Optional: adds nice gradient
)

# Customize layout
fig2.update_traces(
    textposition='outside',
    hovertemplate='<b>%{x}</b><br>Total Volume: %{y:,.2f} USD<extra></extra>'
)

fig2.update_layout(
    yaxis_title='Total USD Volume',
    xaxis_title='Asset Symbol',
    plot_bgcolor='white',
    xaxis_tickangle=-30,
    title_x=0.5,
    uniformtext_minsize=10,
    uniformtext_mode='hide',
)

fig2.show()


###  Daily USD Volume Trend

In [47]:
# convert timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])
# Group by date (daily sum)
daily_volume = df.groupby(df['timestamp'].dt.date)['amount_usd'].sum().reset_index()
daily_volume.columns = ['date', 'total_usd_volume']

import plotly.express as px

fig = px.line(
    daily_volume,
    x='date',
    y='total_usd_volume',
    title='Daily USD Transaction Volume',
    labels={'date': 'Date', 'total_usd_volume': 'Total USD Volume'},
    markers=True
)

fig.update_layout(
    xaxis_title='Date',
    yaxis_title='USD Volume',
    title_x=0.5,
    plot_bgcolor='white'
)

fig.update_traces(
    hovertemplate='Date: %{x}<br>Volume: $%{y:,.2f}<extra></extra>'
)

fig.show()


##### The highest transaction volume was April 24, 2021

### Interactive EDA Time Series by Asset & Time Granularity

In [48]:

import ipywidgets as widgets
from IPython.display import display


# Helper: granularity function
def get_grouped_df(granularity, asset_filter):
    if asset_filter != 'All':
        temp_df = df[df['asset_symbol'] == asset_filter]
    else:
        temp_df = df.copy()

    if granularity == 'Daily':
        grouped = temp_df.groupby(temp_df['timestamp'].dt.date)['amount_usd'].sum().reset_index()
        grouped.columns = ['Time', 'USD Volume']
    elif granularity == 'Weekly':
        grouped = temp_df.groupby(temp_df['timestamp'].dt.to_period('W'))['amount_usd'].sum().reset_index()
        grouped['timestamp'] = grouped['timestamp'].astype(str)
        grouped.columns = ['Time', 'USD Volume']
    else:  # Monthly
        grouped = temp_df.groupby(temp_df['timestamp'].dt.to_period('M'))['amount_usd'].sum().reset_index()
        grouped['timestamp'] = grouped['timestamp'].astype(str)
        grouped.columns = ['Time', 'USD Volume']

    return grouped


##### Widgets for Interactivity


In [49]:
# Dropdowns
granularity_dropdown = widgets.Dropdown(
    options=['Daily', 'Weekly', 'Monthly'],
    value='Daily',
    description='Granularity:',
    style={'description_width': 'initial'}
)

asset_options = ['All'] + sorted(df['asset_symbol'].dropna().unique())
asset_dropdown = widgets.Dropdown(
    options=asset_options,
    value='All',
    description='Asset:',
    style={'description_width': 'initial'}
)


##### Plot Function

In [50]:
# Main plot function
def plot_volume(granularity, asset_filter):
    grouped_df = get_grouped_df(granularity, asset_filter)

    fig = px.line(
        grouped_df,
        x='Time',
        y='USD Volume',
        title=f'{granularity} USD Volume{" for " + asset_filter if asset_filter != "All" else ""}',
        markers=True
    )

    fig.update_layout(
        xaxis_title=granularity,
        yaxis_title='USD Volume',
        title_x=0.5,
        plot_bgcolor='white'
    )

    fig.update_traces(
        hovertemplate='Time: %{x}<br>Volume: $%{y:,.2f}<extra></extra>'
    )

    fig.show()


##### Interactive Plotly Time Series for Assets

In [51]:
# Bind everything together
ui = widgets.HBox([granularity_dropdown, asset_dropdown])
out = widgets.interactive_output(plot_volume, {
    'granularity': granularity_dropdown,
    'asset_filter': asset_dropdown
})

display(ui, out)


Output()